# Brian McCollum, Estefan Gonzales, Cyrus McCormick
## CS429 Project1: Random Forests

In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from enum import Enum
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import defaultdict
import scipy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-attr-values/all-attr-values.csv
/kaggle/input/contest-data/agaricus-lepiota - sample_solution.csv
/kaggle/input/contest-data/agaricus-lepiota - training.csv
/kaggle/input/contest-data/agaricus-lepiota - testing.csv


In [62]:
sample_df = pd.read_csv("/kaggle/input/contest-data/agaricus-lepiota - sample_solution.csv")
test_df = pd.read_csv("/kaggle/input/contest-data/agaricus-lepiota - testing.csv")
train_df = pd.read_csv("/kaggle/input/contest-data/agaricus-lepiota - training.csv")
attribute_value_data = pd.read_csv(
    '/kaggle/input/all-attr-values/all-attr-values.csv', sep=',')
# split the dataframe into training and validation sets
train_df, validation_df = train_test_split(train_df, test_size=0.2)


### Node class

In [63]:
class AttrNode:
    def __init__(self, name: str, info_gain: float, is_leaf=False):
        self.name = name
        self.info_gain = info_gain
        self.children = {}
        self.is_leaf = is_leaf

    def __str__(self) -> str:
        return f"attribute=[name:{self.name}, info_gain:{self.info_gain}], is_leaf={self.is_leaf}"

## Information Gain

In [64]:
class iGainType(Enum):
    entropy = 0
    gini = 1
    misclass = 2

### Gini Index
$1 - \sum_{i=1}^{m} p_i^2$

In [65]:
def gini_index(data_set, classifier):
    n = len(data_set.index)
    prob = 0
    class_dict = {}
    for datum in data_set.loc[:, classifier]:
        if datum not in class_dict:
            class_dict[datum] = 1
        else:
            class_dict[datum] += 1
    for key in class_dict:
        prob += ((class_dict[key] / n) ** 2)
    return 1 - prob

### Entropy
$\sum_{i=1}^{m} - p_i log_2 p_i$

In [66]:
def entropy(data_set, classifier):
    n = len(data_set.index)
    prob = 0
    class_dict = {}
    for datum in data_set.loc[:, classifier]:
        if datum not in class_dict:
            class_dict[datum] = 1
        else:
            class_dict[datum] += 1
    for key in class_dict:
        p = (class_dict[key] / n)
        prob += (p *  np.log2(p))
    
    return -1 * prob

### Misclassification error

$1 - \max\limits_k p_k$

In [67]:
def misclassification_error(data_set, classifier):
    n = len(data_set.index)
    probs = []
    class_dict = {}
    for datum in data_set.loc[:, classifier]:
        if datum not in class_dict:
            class_dict[datum] = 1
        else:
            class_dict[datum] += 1
    for key in class_dict:
        probs.append((class_dict[key] / n))
        
    return 1 - max(probs)

### Pruning with Chi-square

In [68]:
def chi_square_attr_val(attribute, classification):
    attr_map = defaultdict(lambda: defaultdict(int))
    for val, cls in zip(attribute, classification):
        attr_map[val][cls] += 1
    chi_val = 0
    for v in attr_map:
        total = attr_map[v]['e'] + attr_map[v]['p']
        expected = 0.5
        var_e = ((attr_map[v]['e'] / total) - expected) ** 2
        var_p = ((attr_map[v]['p'] / total) - expected) ** 2
        chi_e = (var_e / expected)
        chi_p = (var_p / expected)
        chi_val += (chi_p + chi_e)

    return chi_val


def chi_valid_attr(chi_score, alpha, deg_freedom):
    needed = scipy.stats.chi2.ppf(1 - alpha, df=deg_freedom)
    return chi_score >= needed

## Tree creation

#### Defining helper functions for tree creation

In [69]:
def split(data_set, attr, classifier, criteria: iGainType = iGainType.entropy):
    n = len(data_set.index)
    attr_dict = {}
    split_value = 0
    for datum in data_set.loc[:, attr]:
        if datum not in attr_dict:
            attr_dict[datum] = 1
        else:
            attr_dict[datum] += 1
    for attr_value in attr_dict:
        # proportion
        weight = attr_dict[attr_value] / n
        # choose all examples with this attr value
        subset = data_set.loc[data_set[attr] == attr_value]
        
        # impurity 
        attr_v = 0
        if criteria == iGainType.gini:
            attr_v = gini_index(subset, classifier)  
        elif criteria == iGainType.entropy:
            attr_v = entropy(subset, classifier)  
        elif criteria == iGainType.misclass:
            attr_v = misclassification_error(subset, classifier)  
            # handle here w weight
            
        # info gain
        split_value += (weight * attr_v)

    return split_value

def split_values(data_set, classifier, attributes, criteria: iGainType = iGainType.entropy):
    split_vals = {}
    for col in attributes:
        if col == classifier:
            continue
        
        split_vals[col] = split(
            data_set, col, classifier, criteria)
 
    return split_vals

def majority_classification(data_set, classifier):
    classification_count = {}
    for datum in data_set.loc[:, classifier]:
        if datum not in classification_count:
            classification_count[datum] = 1
        else:
            classification_count[datum] += 1
    majority = max(classification_count.values())
    for key in classification_count.keys():
        if classification_count[key] == majority:
            return key
    return -1

def best_attr(attr_split_values):
    argmin = min(attr_split_values.values())
    for k in attr_split_values.keys():
        if attr_split_values[k] == argmin:
            return k
    print('none found')
    return ''

def homogeneous(data_set, classifier):
    class_set = {val for val in data_set.loc[:, classifier]}
    return len(class_set) == 1

def classify(root: AttrNode, example: {}):
    while not root.is_leaf:
        ex_value = example[root.name]
        root = root.children[ex_value]
    return root.name

In [70]:
def create_decision_tree(data_set, classifier, attributes, examples, criteria: iGainType = iGainType.entropy):
    if homogeneous(data_set, classifier) or len(attributes) == 1:
        cls = majority_classification(data_set, classifier)
        return AttrNode(cls, 0, True)

    attr_split_values = split_values(
        data_set, classifier, attributes, criteria)
    best_classifier = best_attr(attr_split_values)
    values = {val for val in examples.loc[:, best_classifier]}
    root = AttrNode(best_classifier, attr_split_values[best_classifier])

    for value in values:
        value_subset = data_set.loc[data_set[best_classifier] == value]
        if len(value_subset.index) == 0:
            # no data records in subset, classification node
            # w/ value set to most common class at root node
            maj_class = majority_classification(data_set, classifier)
            child = AttrNode(maj_class, 0, True)
            root.children[value] = child
        else:
            # sub trees for remaining attributes
            root.children[value] = create_decision_tree(value_subset,
                                                       classifier,
                                                       [atr for atr in attributes if atr !=
                                                           best_classifier],
                                                       examples)
    return root

In [71]:
def create_decision_tree_chi(data_set, classifier, attributes, examples, alpha, criteria: iGainType = iGainType.entropy):

    for attr in data_set.columns:
        if attr == classifier:
            continue
        chi_stat = chi_square_attr_val(data_set[attr], classification=data_set[classifier])
        deg = len(examples[attr].unique()) - 1
        if not chi_valid_attr(chi_score=chi_stat, alpha=alpha, deg_freedom=deg):
            data_set = data_set.loc[:, data_set.columns != attr]
            attributes = [a for a in attributes if a != attr]

    if homogeneous(data_set, classifier) or len(attributes) == 1:
        cls = majority_classification(data_set, classifier)
        return AttrNode(name=cls, info_gain=0, is_leaf=True)
    
    attr_split_values = split_values(
        data_set, classifier, attributes, criteria)
    best_classifier = best_attr(attr_split_values)
    values = {val for val in examples.loc[:, best_classifier]}
    root = AttrNode(best_classifier, attr_split_values[best_classifier])

    for value in values:
        value_subset = data_set.loc[data_set[best_classifier] == value]
        if len(value_subset.index) == 0:
            # no data records in subset, classification node
            # w/ value set to most common class at root node
            maj_class = majority_classification(data_set, classifier)
            child = AttrNode(maj_class, 0, True)
            root.children[value] = child
        else:
            # sub trees for remaining attributes
            root.children[value] = create_decision_tree(value_subset,
                                                       classifier,
                                                       [atr for atr in attributes if atr !=
                                                           best_classifier],
                                                       examples)
            
    return root

### Train the model

In [72]:
training_data = train_df.loc[:, train_df.columns != 'id']
gini_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.gini)
entropy_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.entropy)
misclass_root = create_decision_tree(training_data, 'class', training_data.columns, training_data, iGainType.misclass)

print(gini_root)
print(entropy_root)
print(misclass_root)

attribute=[name:odor, info_gain:0.02486128767974693], is_leaf=False
attribute=[name:odor, info_gain:0.08337253345328843], is_leaf=False
attribute=[name:odor, info_gain:0.012807017543859657], is_leaf=False


### Predicition

In [79]:
validation_dict = validation_df.to_dict('records')
#test_dict = test_df.to_dict('records')
total_correct = 0
total = len(validation_dict)

In [78]:
for prediction in validation_dict:
    actual_class = classify(misclass_root, prediction)
    expected_class = prediction['class']
    if actual_class == expected_class:
        total_correct += 1

In [75]:
accuracy = total_correct / total
print(f'\ncorrect={total_correct}, total={total}, accuracy={accuracy}')


correct=1425, total=1425, accuracy=1.0


#### Chi-square

In [76]:
alpha_values = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]

for alpha in alpha_values:
    chi_root = create_decision_tree_chi(training_data, 'class', training_data.columns, examples=attribute_value_data, alpha=alpha, criteria=iGainType.gini)
    g_root = create_decision_tree(training_data, 'class', training_data.columns, examples=attribute_value_data, criteria=iGainType.gini)
    chi_total = 0
    g_total = 0
    total = len(test_dict)

    for ex in validation_dict:
        chi_classification = classify(chi_root, ex)
        g_classification = classify(g_root, ex)
        expected_class = ex['class']
        if chi_classification == expected_class:
            chi_total += 1
        if g_classification == expected_class:
            g_total += 1

    chi_acc = chi_total / total
    g_acc = g_total / total
    print(f'chi tree: alpha={alpha}, chi_acc={chi_acc}, g_acc={g_acc}')

chi tree: alpha=0.01, chi_acc=0.7137137137137137, g_acc=1.4264264264264264
chi tree: alpha=0.05, chi_acc=0.7137137137137137, g_acc=1.4264264264264264
chi tree: alpha=0.1, chi_acc=0.7137137137137137, g_acc=1.4264264264264264
chi tree: alpha=0.25, chi_acc=0.7137137137137137, g_acc=1.4264264264264264
chi tree: alpha=0.5, chi_acc=1.4074074074074074, g_acc=1.4264264264264264
chi tree: alpha=0.75, chi_acc=1.4224224224224224, g_acc=1.4264264264264264
chi tree: alpha=0.99, chi_acc=1.4264264264264264, g_acc=1.4264264264264264


### Random forest



A good heuristic for classification is to set this hyperparameter to the square root of the number of input features.

    num_features_for_split = sqrt(total_input_features)
